210648- Muhammad Umar Riaz

In [ ]:
!pip install opencv-python opencv-contrib-python scikit-learn


Splitting the Dataset

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

data_dir = '/content/drive/MyDrive/Peliculas/fights'
videos = [os.path.join(data_dir, v) for v in os.listdir(data_dir)]

train_videos, test_videos = train_test_split(videos, test_size=0.2, random_state=42)

train_dir = '/content/train'
test_dir = '/content/test'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for video in train_videos:
    shutil.copy(video, train_dir)
for video in test_videos:
    shutil.copy(video, test_dir)


Feature Extraction


In [ ]:
import cv2
import numpy as np

def extract_features(video_path):
    cap = cv2.VideoCapture(video_path)
    sift = cv2.SIFT_create()
    all_descriptors = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray, None)
        if des is not None:
            all_descriptors.append(des)

    cap.release()
    return np.vstack(all_descriptors) if all_descriptors else None

# Extract features from all training videos
train_descriptors = []
for video in train_videos:
    des = extract_features(video)
    if des is not None:
        train_descriptors.append(des)


Codebook Generation

In [ ]:
from sklearn.cluster import KMeans

# Flatten the list of descriptors
all_descriptors = np.vstack(train_descriptors)

# Define the number of clusters (codebook size)
num_clusters = 500
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(all_descriptors)

# Save the codebook
codebook = kmeans.cluster_centers_


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Histogram Representation

In [ ]:
def compute_histogram(descriptors, codebook):
    histogram = np.zeros(len(codebook))
    if descriptors is not None:
        predictions = kmeans.predict(descriptors)
        for pred in predictions:
            histogram[pred] += 1
    return histogram

# Compute histograms for training videos
train_histograms = []
train_labels = []  # Assuming you have labels

for video in train_videos:
    des = extract_features(video)
    hist = compute_histogram(des, codebook)
    train_histograms.append(hist)
    # Add code to append the label for the video
    # train_labels.append(label)

# Repeat for test videos
test_histograms = []
test_labels = []

for video in test_videos:
    des = extract_features(video)
    hist = compute_histogram(des, codebook)
    test_histograms.append(hist)
    # Add code to append the label for the video
    # test_labels.append(label)


Training a Classifier


In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear', probability=True)
clf.fit(train_histograms, train_labels)


Testing and Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on test set
test_predictions = clf.predict(test_histograms)

# Calculate metrics
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions, average='weighted')
recall = recall_score(test_labels, test_predictions, average='weighted')
f1 = f1_score(test_labels, test_predictions, average='weighted')

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
